## <center>SOCCER PLAYER ASSIST PREDICTION REGRESSION ANALYSIS<center>

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

In [2]:

# Load dataset
df_raw = pd.read_csv('501_PROJECT_DATASET.csv')

# Remove rows where 'GOAL_SCORER' or 'GOAL_SCORER_TEAM_NAME' is null
df_raw = df_raw.dropna(subset=['GOAL_SCORER', 'GOAL_SCORER_TEAM_NAME'])

# Drop unnecessary columns
df_cleaned = df_raw.drop(columns=['DURATION', 'SEASON', 'GOAL_DESC', 'DATE_TIME', 'FOOT', 'POSITION', 
                                  'HEIGHT', 'WEIGHT', 'GOAL_SCORER', 'GOALS'])

df_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2049 entries, 0 to 2337
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   HOME_TEAM              2049 non-null   object
 1   AWAY_TEAM              2049 non-null   object
 2   STADIUM                2049 non-null   object
 3   GOAL_SCORER_TEAM_NAME  2049 non-null   object
 4   ASSIST_PLAYER          1466 non-null   object
 5   ASSISTS                2049 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 112.1+ KB


In [3]:
# Categorical columns for one-hot encoding
categorical_cols = ['HOME_TEAM', 'AWAY_TEAM', 'STADIUM', 'ASSIST_PLAYER', 'GOAL_SCORER_TEAM_NAME']

# Load dataset and preprocessing as before...

# Separate features and target variables for both 'GOALS' and 'ASSISTS'
X = df_cleaned.drop(columns=['ASSISTS'])  # Features
y = df_cleaned[['ASSISTS']]  # Target variables for both 'GOALS' and 'ASSISTS'

# Split the data into training and testing sets for both features and targets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define a preprocessing pipeline for one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

In [4]:
# Create a dictionary of regressors for different algorithms
regressors = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet Regression': ElasticNet(),
    'Random Forest Regression': RandomForestRegressor(),
    'SVR': SVR(),
    'XGBoost Regression': XGBRegressor()
}

# Fit and evaluate each regressor with MultiOutputRegressor
for name, regressor in regressors.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', regressor)
    ])

    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    mse_goals = mean_squared_error(y_test, predictions)
    print(f"{name} MSE for ASSISTS: {mse_goals}")


Linear Regression MSE for ASSISTS: 0.20887291625996382
Ridge Regression MSE for ASSISTS: 0.19056551311127715
Lasso Regression MSE for ASSISTS: 0.6713629723400562
ElasticNet Regression MSE for ASSISTS: 0.6713629723400562


C:\Users\dalvi\.conda\envs\InfoSys501General\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Regression MSE for ASSISTS: 0.09244707317073171


C:\Users\dalvi\.conda\envs\InfoSys501General\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR MSE for ASSISTS: 0.16402422190581606
XGBoost Regression MSE for ASSISTS: 0.09878874268001099


In [5]:


# Define the parameter grid to search
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [5, 9, 10],
    'regressor__learning_rate': [0.01, 0.1, 0.2]
}

# Create a pipeline with preprocessing and XGBoost Regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

# Apply RandomizedSearchCV
random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)

# Get the best parameters
best_params = random_search.best_params_
print("Best Parameters:", best_params)

# Predict on the test set with the best model
best_model = random_search.best_estimator_
predictions = best_model.predict(X_test)

# Calculate Mean Squared Error (MSE) for both goals and assists
mse_goals = mean_squared_error(y_test, predictions)
print(f"Tuned XGBoost Regression MSE for ASSISTS: {mse_goals}")

Best Parameters: {'regressor__n_estimators': 300, 'regressor__max_depth': 5, 'regressor__learning_rate': 0.1}
Tuned XGBoost Regression MSE for ASSISTS: 0.10573751919132743


In [6]:

# Use the best parameters obtained from the hyperparameter tuning
best_params = {
    'n_estimators': 450,
    'max_depth': 10,
    'learning_rate': 0.2
}

# Create an XGBoost regressor with the best parameters
best_model = XGBRegressor(**best_params)

# Create a pipeline with preprocessing and XGBoost Regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', best_model)
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict on the test set
predictions = pipeline.predict(X_test)

# Calculate Mean Squared Error (MSE) for both goals and assists
mse_goals = mean_squared_error(y_test, predictions)
print(f"Tuned XGBoost Regression MSE for ASSISTS: {mse_goals}")


Tuned XGBoost Regression MSE for ASSISTS: 0.10766809903853986


In [7]:
# xgboost is much faster than the random forest 
# hence we are finalising our model and choosing xgboost for better mse and faster prediction

# Sample data for a single row (adjust according to your actual data)
input_data = {
    'HOME_TEAM': 'Real Madrid',
    'AWAY_TEAM': 'SSC Napoli',
    'STADIUM': 'Santiago Bernabéu',
    'ASSIST_PLAYER': 'David Alaba',
    'GOAL_SCORER_TEAM_NAME': 'Real Madrid'
}

# Create a DataFrame from the single row data
input_df = pd.DataFrame([input_data])

# Use the trained pipeline to preprocess the single row data and make predictions
predictions = pipeline.predict(input_df)

print(f"Predicted Assist : {predictions}")



Predicted Assist : [1.0901728]


In [8]:
import joblib

# Save the trained model to a file
joblib.dump(pipeline, 'xgboost_assists_model.pkl')

['xgboost_assists_model.pkl']